### Automação do SUAP-IFPE versão 0.1

#### Importando webdriver e abrindo SUAP

In [ ]:
import pandas as pd

tombos_df = pd.read_excel('biblioteca_sala_estudos.xlsx')
#display(tombos_df)

tombo_impressao = []
empenho_impressao = []
uasg_impressao = []
nf_impressao = []
dtnf_impressao = []
qrcode_impressao = []

carga_impressao = []
descricao_impressao = []

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time



def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys("2184579")

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys("sis1986%")

    nav.find_element_by_xpath('//*[@id="ok"]').click()

    
# abrir_suap()


In [ ]:
def trocar_sala():
    
    abrir_suap()
    
    for linha in tombos_df["tombos"]:
        nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

        tombo_impressao.append(linha)
        print(linha)

        carga = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[7]/td[2]").get_attribute('innerText')
        carga_impressao.append(carga)
        print(carga)

        descricao = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td[2]/span[1]").get_attribute('innerText')
        descricao_impressao.append(descricao)
        print(descricao)

        valor_sala_suap = nav.find_element_by_xpath('//*[@id="id_sala"]').get_attribute('value')
        valor_nova_sala = '1990'

        # time.sleep(1)

        if valor_sala_suap != valor_nova_sala:
            nova_sala = Select(nav.find_element_by_id('id_sala'))
            nova_sala.select_by_value(valor_nova_sala)
            time.sleep(1)
            nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[2]/input').click()
    
    nav.quit()

trocar_sala()

In [ ]:
def impressao_etiqueta():
    
    abrir_suap()
    
    for linha in tombos_df["tombos"]:
    
        nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")
        # print(linha)
        nav.find_element_by_link_text("Detalhes da Entrada").click()
    
        empenho = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
        empenho = empenho[:12]
        empenho_impressao.append(empenho)
        # print(empenho)

        uasg = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
        uasg_impressao.append(uasg)
        # print(uasg)

        nf = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
        nf_impressao.append(nf)
        # print(nf)

        data = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
        dtnf_impressao.append(data)
        # print(data)

        qrcode = str(linha)+" "+empenho+" "+" "+uasg+" "+nf+" "+data
        qrcode_impressao.append(qrcode)
        
    nav.quit()
    
impressao_etiqueta()


In [ ]:
def montar_impressao_etiqueta():
    tabela_impressao = pd.DataFrame()
    tabela_impressao['patrimonio'] = tombo_impressao
    tabela_impressao['empenho'] = empenho_impressao
    tabela_impressao['uasg'] = uasg_impressao
    tabela_impressao['nota fiscal'] = nf_impressao
    tabela_impressao['data nota fiscal'] = dtnf_impressao
    tabela_impressao['qrcode'] = qrcode_impressao
    display(tabela_impressao)

montar_impressao_etiqueta()


In [ ]:
def montar_tabela_carga():
    tabela_carga = pd.DataFrame()
    tabela_carga['tombo'] = tombo_impressao
    tabela_carga['descricao'] = descricao_impressao
    tabela_carga['carga'] = carga_impressao

    display(tabela_carga)

montar_tabela_carga()